## Week 4 — Streamlit Dashboard for Aircraft Engine RUL Prediction

In Week 4, we convert our trained LSTM/GRU deep learning models into a real, interactive **Predictive Maintenance Dashboard** using Streamlit.

This dashboard allows:

- Uploading engine sensor data  
- Running preprocessing (scaling + sequence creation)  
- Predicting Remaining Useful Life (RUL)  
- Displaying engine health status (Green/Yellow/Red)  
- Showing model outputs in real-time  

It simulates an airline maintenance control panel used by Emirates, Qatar Airways, and Ethiopian Airlines.
definedtes-style).



## Step 1 — Import Required Libraries

We import all necessary libraries for:

- Data preprocessing  
- Sequence generation  
- TensorFlow model loading  
- RUL prediction  
- Streamlit dashboard integration  


In [11]:
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## Step 2 — Load FD001 Test Data

We load the test dataset using the same column structure as Week 1.  
The test dataset contains incomplete engine run cycles (ends before failure).  

The true RUL for each engine is provided separately in RUL_FD001.txt.

In [18]:
BASE_DIR = Path(r"C:\Users\Kal\Predictive Maintenance Aircraft Engine")
DATA_DIR = BASE_DIR / "data" / "raw"

col_names = ["engine_id", "cycle",
             "setting_1", "setting_2", "setting_3"] + \
            [f"sensor_{i}" for i in range(1, 22)]

test_df = pd.read_csv(DATA_DIR / "test_FD001.txt", sep=r"\s+", header=None, names=col_names)
test_df.head()


,engine_id,cycle,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


## Step 3 — Define the Same Feature Columns Used During Training

Deep learning models require **exactly the same input features**  
(total: 12 features) that were used during training in Week 3.

These include:
- 9 selected sensors showing degradation patterns  
- 3 operating settings  

In [19]:
useful_sensors = [
    "sensor_2", "sensor_3", "sensor_4",
    "sensor_7", "sensor_8",
    "sensor_11", "sensor_12", "sensor_13", "sensor_14"
]

feature_columns = useful_sensors + ["setting_1", "setting_2", "setting_3"]
len(feature_columns)


12

## Step 4 — Normalize Test Data Using MinMax Scaling

We apply MinMax scaling to the test data using the same feature set.  
Since test data has no training statistics, we fit the scaler directly on the test set.

This ensures:
- Input values stay in the 0–1 range  
- Model receives inputs in the same scale as during training  

In [20]:
scaler = MinMaxScaler()
test_scaled = test_df.copy()
test_scaled[feature_columns] = scaler.fit_transform(test_df[feature_columns])

test_scaled[feature_columns].head()


,sensor_2,sensor_3,sensor_4,sensor_7,sensor_8,sensor_11,sensor_12,sensor_13,sensor_14,setting_1,setting_2,setting_3
0,0.596215,0.421968,0.282214,0.608871,0.365854,0.273973,0.534247,0.325581,0.152259,0.65625,0.692308,0.0
1,0.182965,0.504025,0.225240,0.800403,0.292683,0.479452,0.634703,0.395349,0.277907,0.34375,0.230769,0.0
2,0.419558,0.464814,0.346130,0.651210,0.390244,0.479452,0.591324,0.325581,0.192892,0.53125,0.538462,0.0
3,0.413249,0.391587,0.449867,0.643145,0.341463,0.328767,0.456621,0.372093,0.217896,0.77500,0.461538,0.0
4,0.435331,0.471306,0.357974,0.661290,0.292683,0.349315,0.632420,0.325581,0.187891,0.60000,0.461538,0.0


## Step 5 — Create Sliding Window Sequences for Prediction

Our LSTM model requires **30-cycle sequences**.

So we generate:
- Overlapping windows of shape `(30, 12)`  
- One prediction per window  
- The last window represents the most recent engine state  
python
Copy code


In [21]:
SEQ_LEN = 30

def make_sequences(df, features, seq_len=SEQ_LEN):
    X = []
    values = df[features].values
    for i in range(len(df) - seq_len + 1):
        X.append(values[i:i+seq_len])
    return np.array(X)

X_sequences = make_sequences(test_scaled, feature_columns)
X_sequences.shape


(13067, 30, 12)

## Step 6 — Load Trained LSTM/GRU Model

We load the best-performing deep learning model saved in Week 3.

The model expects input shape: `(None, 30, 12)`.

In [22]:
MODEL_PATH = BASE_DIR / "models" / "lstm_fd001_best.h5"
model = tf.keras.models.load_model(MODEL_PATH)
model


## Step 7 — Predict Remaining Useful Life (RUL)

We run model predictions on the test sequences.

Each sliding window yields one RUL prediction.
The **last sequence** corresponds to the engine’s most current condition.

In [23]:
rul_pred = model.predict(X_sequences)
rul_pred.shape


409/409 [==============================] - 1s 2ms/step


(13067, 1)

## Step 8 — Extract Final RUL Prediction

The most relevant prediction is the **last one**,  
because it represents the engine’s most recent 30-cycle time window.

In [25]:
predicted_rul = float(rul_pred[-1])
print("Predicted RUL (cycles):", predicted_rul)

Predicted RUL (cycles): 3.401665210723877


## Step 9 — Convert RUL to Engine Health Status

We classify engine status using traffic-light rules:

- 🟢 **Healthy** → RUL > 50  
- 🟡 **Warning** → 20 < RUL ≤ 50  
- 🔴 **Critical** → RUL ≤ 20  
python
Copy code


In [26]:
def get_engine_status(rul):
    if rul > 50:
        return "🟢 Healthy — No maintenance needed"
    elif rul > 20:
        return "🟡 Warning — Schedule maintenance"
    else:
        return "🔴 Critical — Maintenance required now"

print("Engine Status:", get_engine_status(predicted_rul))


Engine Status: 🔴 Critical — Maintenance required now


# Week 4 Summary

In this week, we completed:

### ✅ Preprocessing of test data for real-time predictions  
### ✅ Sliding window generation for LSTM compatibility  
### ✅ Loading LSTM model from Week 3  
### ✅ Predicting the Remaining Useful Life (RUL)  
### ✅ Converting RUL into an engine health status  
### ✅ Building a Streamlit-ready workflow  

Next, we will create `app.py` to turn this pipeline into a full **interactive dashboard**.